In [1]:
try:
    from google.colab import drive

    drive.mount('./drive/')

    %cd drive/My \ Drive/Text_Summarization

except:
    print("No Colab Environment")

Mounted at ./drive/
/content/drive/My  Drive/Text_Summarization


In [2]:
creds = {"API_Key":"zE9ZTAtXkVOjKMThFK8sRGYa0pKnqH-e5bH7q-usws_f",
"Service_URL":"https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/1787df31-35bf-481a-a6de-562efb73f4d3"
}


api_key = creds['API_Key']
service_url = creds['Service_URL']

In [3]:
!pip install ibm_watson

     |████████████████████████████████| 389kB 5.3MB/s 
     |████████████████████████████████| 204kB 7.7MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-5.1.0-cp37-none-any.whl size=375439 sha256=9b3b80843f8488f435065973f0238670ca08ccdba79a9177fefdab1714aa8555
  Stored in directory: /root/.cache/pip/wheels/49/6d/cf/1d91261b96363da78bf9b02699fd2262e6b5dad179500690c1
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.9.0-cp37-none-any.whl size=59612 sha256=f49ee2343cb60d115be3366da54cb734298ea8b7c708405bc1523371856cd331
  Stored in directory: /root/.cache/pip/wheels/a2/23/f2/ae9db79b4234ed0fc74bf00bc97bcd16440ab2764c5f443167
Successfully built ibm-watson ibm-cloud-sdk-core


In [4]:
import pandas as pd

df = pd.read_csv("Data/processed/Telehealth.csv")
df.head()

,Journal Title,Article Title,Date Published,Authors,Abstract,Keywords,Citation,Content,Content_Length,Abstract_Length,Parsed_Keywords,Parsed_Keywords_Length,Subfield,et_al_Count,Stopwords_Lemma_Longform_Clean_Content,Clean_Content,Classification
0,Psychological Services,The Effectiveness of Telepsychology With Veter...,2021,Michael J. McClellan; Richard Osbaldiston; Ron...,Veterans face a variety of stressors due to th...,"KEYWORDS:\n\ntelepsychology, meta-analysis, ve...","McClellan, M. J., Osbaldiston, R., Wu, R., Yea...",Veterans face a variety of stressors related t...,37477,2411,"['telepsychology,', 'meta-analysis,', 'veteran...",5,Clinical & Counseling Psychology,50,veteran face variety stressor relate military ...,veteran face variety stressor relate military ...,Covid
1,"Psychology, Public Policy, and Law",Making the Case for Videoconferencing and Remo...,2021,Milfred D. Dale; Desiree Smith,The COVID-19 pandemic and its requirements for...,"KEYWORDS:\n\nremote child custody evaluations,...","Dale, M. D., & Smith, D. (2021). Making the ca...",The impact of the COVID-19 pandemic on the chi...,74025,1577,"['remote', 'child', 'custody', 'evaluations,',...",10,Forensic Psychology,48,impact coronavirus disease 2019 pandemic child...,impact coronavirus disease 2019 pandemic child...,Covid
2,"Psychology, Public Policy, and Law","Forensic E-Mental Health: Review, Research Pri...",2021,Lauren E. Kois; Jennifer Cox; Ashley T. Peck,Forensic e-mental health is an area of psychol...,"KEYWORDS:\n\nforensic e-mental health, telehea...","Kois, L. E., Cox, J., & Peck, A. T. (2021). Fo...","In November 2019, COVID-19—the infectious, hig...",63568,1842,"['forensic', 'e-mental', 'health,', 'telehealt...",8,Forensic Psychology,128,November 2019 coronavirus disease 2019 infecti...,November 2019 coronavirus disease 2019 infecti...,Covid
3,Training and Education in Professional Psychology,Moving Toward a New Era of Telepsychology in U...,2021,Allison L. Baier; Sarah Danzo,Many university training clinics are facing nu...,"KEYWORDS:\n\neducation and training, telepsych...","Baier, A. L., & Danzo, S. (2021). Moving towar...",Anxiety and depressive disorders are among the...,30147,1650,"['education', 'and', 'training,', 'telepsychol...",6,Clinical & Counseling Psychology,21,anxiety depressive disorder common impair ment...,anxiety depressive disorder common impair ment...,Covid
4,Practice Innovations,Navigating Changes in the Physical and Psychol...,2021,Liat Shklarski; Allison Abrams; Elana Bakst,The emergence of the Covid-19 pandemic at the ...,"KEYWORDS:\n\nremote psychotherapy, Covid-19, p...","Shklarski, L., Abrams, A., & Bakst, E. (2021)....","In mid-March, 2020, the authors—as well as the...",42367,1871,"['remote', 'psychotherapy,', 'Covid-19,', 'pan...",6,Clinical & Counseling Psychology,14,mid March 2020 author majority therapist advis...,mid March 2020 author majority therapist advis...,Covid


In [5]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, SentimentOptions,EntitiesOptions

authenticator = IAMAuthenticator(api_key)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

natural_language_understanding.set_service_url(service_url)

for i in range(len(df["Clean_Content"])):

    corpus = df["Clean_Content"].iloc[i]

    response = natural_language_understanding.analyze(
        text=corpus,
        features=Features(entities=EntitiesOptions()
        
        )
        
        ).get_result()

    with open(f"Data/processed/IBM/ibm_ner_analysis_{i}.json","w") as f:

        json.dump(response,f,indent=4)



In [6]:
df_ibm = pd.DataFrame.from_dict(response["entities"])

In [7]:
df_ibm.head()

,type,text,relevance,count,confidence,disambiguation
0,Person,Comer Barlow,0.951857,3,0.999836,NaN
1,Person,Eyberg Funderburg,0.818056,2,0.623222,NaN
2,Person,Kazdin Blase,0.678095,3,0.955832,NaN
3,Person,Guy Bonato,0.571831,2,0.864625,NaN
4,HealthCondition,disorder disruptive behavior disorders common ...,0.511488,1,0.071707,NaN


In [8]:
df_ibm = df_ibm.rename(columns={"text":"Term"})

In [9]:
with open("Data/processed/IBM/ibm_ner_analysis_0.json") as f:

    data = json.load(f)
    
    df = pd.DataFrame.from_dict(data["entities"])


In [17]:
import pandas as pd
import glob

import json

ibm_files = glob.glob("Data/processed/IBM/*.json")

global_df = pd.DataFrame()


for i,ibm_file in enumerate(ibm_files):

    with open(ibm_file) as f:

        data = json.load(f)

        df = pd.DataFrame.from_dict(data["entities"])

        df["Document"] = i

        global_df = pd.concat([global_df,df])

        





In [19]:
global_df.head()

,type,text,relevance,count,confidence,disambiguation,Document
0,Person,Bolton Dorstyn,0.953405,6,0.970710,NaN,0
1,HealthCondition,trauma depression,0.834730,8,0.994297,NaN,0
2,Person,Lazar,0.772091,5,0.999862,NaN,0
3,Person,Luxton Nelson,0.765374,3,0.927518,NaN,0
4,HealthCondition,symptom depression,0.681805,3,0.603748,NaN,0


In [22]:
global_df.describe()

,relevance,count,confidence,Document
count,1781.000000,1781.000000,1781.000000,1781.000000
mean,0.290700,1.643459,0.574338,21.070747
std,0.212394,2.658524,0.278756,12.930086
min,0.005777,1.000000,0.024269,0.000000
25%,0.134282,1.000000,0.351339,10.000000
50%,0.245401,1.000000,0.560715,20.000000
75%,0.370150,1.000000,0.826971,34.000000
max,0.973775,67.000000,1.000000,43.000000


In [23]:
global_df["Document"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43])

In [27]:
global_df.to_json("Data/processed/ibm_ner_analysis.json",orient="records")